## Preamble

In [ ]:
%load_ext autoreload

In [ ]:
import os as _os

_os.chdir(_os.environ["PROJECT_ROOT"])
_os.path.realpath(_os.path.curdir)

### Imports

In [ ]:
import os
import subprocess
import time
from itertools import chain, product
from tempfile import mkstemp
from warnings import filterwarnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sfacts as sf
import statsmodels.formula.api as smf
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable

# from fastcluster import linkage
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist, squareform
from statsmodels.graphics.regressionplots import influence_plot
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

import lib.plot
import lib.thisproject.data
from lib.pandas_util import align_indexes, aligned_index, idxwhere, invert_mapping

In [ ]:
sns.set_context("talk")
plt.rcParams["figure.dpi"] = 100

In [ ]:
def _calculate_2tailed_pvalue_from_perm(obs, perms):
    hypoth_left = perms > obs
    hypoth_right = perms < obs
    null_p_left = (hypoth_left.sum() + 1) / (len(hypoth_left) + 1)
    null_p_right = (hypoth_right.sum() + 1) / (len(hypoth_right) + 1)
    return np.minimum(null_p_left, null_p_right) * 2

In [ ]:
def linkage_order(linkage, labels):
    return labels[sp.cluster.hierarchy.to_tree(linkage).pre_order(lambda x: x.id)]


def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


def iterate_primes_up_to(n, return_index=False):
    n = int(np.ceil(n))
    idx = 0
    for i in range(n):
        if is_prime(i):
            if return_index:
                yield (idx, i)
            else:
                yield i
            idx += 1


def maximally_shuffled_order(sorted_order):
    n = len(sorted_order)
    primes_list = list(iterate_primes_up_to(np.sqrt(n)))
    table = pd.DataFrame(np.arange(n), index=sorted_order, columns=["original_order"])
    for prime in primes_list:
        table[prime] = table.original_order % prime
    table.sort_values(primes_list).original_order.values
    table = table.assign(new_order=table.sort_values(primes_list).original_order.values)
    z = table.sort_values("new_order").original_order.values
    table["delta"] = [np.nan] + list(z[1:] - z[:-1])
    return table.sort_values("new_order").index.to_list()

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression


def calc_cross_correlation(x, y, metric='correlation'):
    cc = pd.DataFrame(
        1
        - sp.spatial.distance.cdist(
            x, y, metric=metric
        ),
        index=x.index,
        columns=y.index,
    )
    return cc, None

def pls_pseudo_mapping(x, y, scale=False, tol=1e-7, **kwargs):
    pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=scale, tol=tol, max_iter=5000).fit(
        x.T, y.T
    )
    pseudo_map = pd.DataFrame(
        (pls.y_rotations_ @ pls.x_rotations_.T), index=y.index, columns=x.index
    )
    return pseudo_map.T, pls


def pls_coefs(x, y, scale=False, tol=1e-7, **kwargs):
    pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=scale, tol=tol, max_iter=5000).fit(
        x.T, y.T
    )
    coefs = pd.DataFrame(
        (pls.coef_), index=y.index, columns=x.index
    )
    return coefs.T, pls

def cross_covar(x, y):
    x_centered = x - x.mean()
    y_centered = y - y.mean()
    cc = x @ y.T
    return cc, None

## Construct Metadata

In [ ]:
pair_type_palette={'Transition': 'plum', 'EEN': 'pink', 'PostEEN': 'lightblue'}

diet_palette = {
    "EEN": "lightgreen",
    "PostEEN": "lightblue",
    "InVitro": "plum",
    "PreEEN": "lightpink",
}

subject_order = [
    "A",
    "B",
    "H",
    "C",
    "D",
    "E",
    "F",
    "G",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
]

# NOTE: Requires a dummy value because I want exactly 20 items.
subject_palette = lib.plot.construct_ordered_palette(
    subject_order + [f"dummy{i}" for i in range(20 - len(subject_order))], cm="tab20"
)
subject_palette["X"] = "black"
pair_type_order = ["EEN", "Transition", "PostEEN"]
pair_type_marker_palette = {"EEN": "s", "Transition": ">", "PostEEN": "o"}
pair_type_linestyle_palette = {"EEN": ":", "Transition": "-.", "PostEEN": "-"}

In [ ]:
sample = (
    pd.read_table("meta/een-mgen/sample.tsv")
    .assign(
        label=lambda x: x[
            ["collection_date_relative_een_end", "diet_or_media", "sample_id"]
        ].apply(tuple, axis=1)
    )
    .set_index("sample_id")
)
subject = pd.read_table("meta/een-mgen/subject.tsv", index_col="subject_id")

In [ ]:
rotu_counts = pd.read_table(
    "data/group/een/a.proc.zotu_counts.tsv", index_col="#OTU ID"
).rename_axis(index="zotu", columns="sample_id")
rotu_taxonomy0 = rotu_counts.taxonomy
rotu_counts = rotu_counts.drop(columns=["taxonomy"]).T
rotu_rabund = rotu_counts.divide(rotu_counts.sum(1), axis=0)

sample_rotu_bc_linkage = sp.cluster.hierarchy.linkage(
    rotu_rabund, method="average", metric="braycurtis", optimal_ordering=True
)

In [ ]:
rotu_taxonomy = rotu_taxonomy0.str.split(';').apply(lambda x: pd.Series(x, index=['d__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']))

In [ ]:
missing_samples = sorted(idxwhere(~rotu_counts.index.to_series().isin(sample.index)))
print(len(missing_samples), ", ".join(missing_samples))

In [ ]:
x = rotu_rabund
row_colors = pd.DataFrame(
    dict(
        subj=sample.subject_id.map(subject_palette),
        swap=sample.index.to_series()
        .isin(["CF_11", "CF_15"])
        .replace({False: "grey", True: "black"}),
    )
)
row_linkage = sample_rotu_bc_linkage

sns.clustermap(
    rotu_rabund,
    norm=mpl.colors.PowerNorm(1 / 5),
    row_colors=row_colors,
    row_linkage=row_linkage,
)

In [ ]:
gtpro_depth = (pd.read_table(
    "data/group/een/r.proc.gtpro.all_species_depth.tsv",
    index_col=['sample', "species_id"],
    )
    .depth.unstack(fill_value=0)
    .rename(columns=str, index=lambda x: "CF_" + str(int(x.split("_")[1])))
    .rename({'CF_15': 'CF_11', 'CF_11': 'CF_15'})  # Sample swap
)
gtpro_rabund = gtpro_depth.divide(gtpro_depth.sum(1), axis=0)

gtpro_rabund

In [ ]:
motu_depth = (pd.read_table(
    "data/group/een/r.proc.gene99_v20-v23-agg75.spgc_specgene-ref-filt-p95.all_species_depth.tsv",
    index_col=['sample', "species_id"],
    )
    .depth.unstack(fill_value=0)
    .rename(columns=str, index=lambda x: "CF_" + str(int(x.split("_")[1])))
    .rename({'CF_15': 'CF_11', 'CF_11': 'CF_15'})  # Sample swap
)
motu_rabund = motu_depth.divide(motu_depth.sum(1), axis=0)

sample_motu_bc_linkage = sp.cluster.hierarchy.linkage(
    motu_rabund, method="average", metric="braycurtis", optimal_ordering=True
)

motu_rabund

In [ ]:
x, y = align_indexes(motu_rabund, rotu_rabund)


x_linkage = linkage(x, method="average", metric="braycurtis", optimal_ordering=True)
y_linkage = linkage(y, method="average", metric="braycurtis", optimal_ordering=True)
colors = pd.DataFrame(
    dict(
        subj=sample.subject_id.map(subject_palette),
        swap=sample.index.to_series()
        .isin(["CF_11", "CF_15"])
        .replace({False: "grey", True: "black"}),
    )
)

x_pdist = pd.DataFrame(
    squareform(pdist(x, metric="braycurtis")), index=x.index, columns=x.index
)
sns.clustermap(
    x_pdist,
    row_linkage=y_linkage,
    col_linkage=x_linkage,
    row_colors=colors,
    col_colors=colors,
)

In [ ]:
x, y = align_indexes(motu_rabund, gtpro_rabund)


x_linkage = linkage(x, method="average", metric="braycurtis", optimal_ordering=True)
y_linkage = linkage(y, method="average", metric="braycurtis", optimal_ordering=True)
colors = pd.DataFrame(
    dict(
        subj=sample.subject_id.map(subject_palette),
        swap=sample.index.to_series()
        .isin(["CF_11", "CF_15"])
        .replace({False: "grey", True: "black"}),
    )
)

x_pdist = pd.DataFrame(
    squareform(pdist(x, metric="braycurtis")), index=x.index, columns=x.index
)
sns.clustermap(
    x_pdist,
    row_linkage=y_linkage,
    col_linkage=x_linkage,
    row_colors=colors,
    col_colors=colors,
)

In [ ]:
bins = np.linspace(0, 30_000, num=200)

fig, axs = plt.subplots(2, sharex=True)

for (title, x), ax in zip(
    dict(
        total_depth_by_sample=motu_depth.sum(1),
        total_depth_by_species=motu_depth.sum(0),
    ).items(),
    axs.flatten(),
):
    ax.hist(x, bins=np.logspace(-1, 5, num=100))
    ax.set_title(title)
    ax.set_xscale("log")
fig.tight_layout()

In [ ]:
motu_rabund.mean().sort_values(ascending=False).head(20)

In [ ]:
n_species = 10
top_motus = (
    (motu_rabund > 1e-5).sum().sort_values(ascending=False).head(n_species).index
)

fig, axs = plt.subplots(
    n_species, figsize=(5, 0.3 * n_species), sharex=True, sharey=True
)

bins = np.logspace(-8, 1, num=51)

for species_id, ax in zip(top_motus, axs):
    # ax.hist(rabund_subset[species_id], bins=bins, alpha=0.7)
    ax.hist(motu_rabund[species_id], bins=bins, alpha=0.7)
    ax.set_xscale("log")
    prevalence = (motu_rabund[species_id] > 1e-5).mean()
    ax.set_title("")
    # ax.set_xticks()
    # ax.set_yticks()
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.patch.set_alpha(0.0)
    for spine in ["left", "right", "top", "bottom"]:
        ax.spines[spine].set_visible(False)
    ax.annotate(
        f"{species_id} ({prevalence:0.0%})",
        xy=(0.05, 0.1),
        ha="left",
        xycoords="axes fraction",
    )
    ax.set_xlim(left=1e-9)
    ax.set_ylim(top=20)
    ax.axvline(1e-5, lw=1, linestyle=":", color="k")

ax.xaxis.set_visible(True)
ax.spines["bottom"].set_visible(True)
ax.set_xticks([1e-4, 1e-2, 1e-0])
ax.set_xticklabels(["0.01%", "1%", "100%"])
ax.set_xlabel("Relative Abundance")

# fig.subplots_adjust(hspace=-0.75)

In [ ]:
def parse_taxonomy_string(taxonomy_string):
    values = taxonomy_string.split(";")
    return pd.Series(values, index=["d__", "p__", "c__", "o__", "f__", "g__", "s__"])

In [ ]:
motu_taxonomy_inpath = "ref/uhgg_genomes_all_v2.tsv"

_motu_taxonomy = (
    pd.read_table(motu_taxonomy_inpath)[lambda x: x.Genome == x.Species_rep]
    .assign(species_id=lambda x: "1" + x.MGnify_accession.str.split("-").str[2])
    .set_index("species_id")
)

# motu_lineage_string = _motu_taxonomy.Lineage

motu_taxonomy = _motu_taxonomy.Lineage.apply(
    parse_taxonomy_string
)  # .assign(taxonomy_string=motu_lineage_string)
motu_taxonomy

In [ ]:
for _species_id in top_motus.astype(str):
    print(_species_id, ":", ";".join(motu_taxonomy.loc[_species_id].values))

## Statistical Linkage Between zOTUs and GT-Pro Species

### "Enterobacteriaceae"

#### Zotu4

In [ ]:
focal_zotu = ["Zotu4"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### 102351

In [ ]:
focal_motu = '102351'

(
    d.reorder_levels(["species_id", "zotu"])
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .join(rotu_taxonomy0.str[-20:])
    .loc[focal_motu]
)

### "Eggerthellaceae" (Zotu172)

In [ ]:
focal_zotu = ["Zotu172"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Lachnospiraceae"

#### Zotu114

In [ ]:
focal_zotu = ["Zotu114"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu25

In [ ]:
focal_zotu = ["Zotu25"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu5

In [ ]:
focal_zotu = ["Zotu5"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu10

In [ ]:
focal_zotu = ["Zotu10"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu160

In [ ]:
focal_zotu = ["Zotu160"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu31

In [ ]:
focal_zotu = ["Zotu31"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu155

In [ ]:
focal_zotu = ["Zotu155"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu75

In [ ]:
focal_zotu = ["Zotu75"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu13

In [ ]:
focal_zotu = ["Zotu13"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu155

In [ ]:
focal_zotu = ["Zotu155"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu27

In [ ]:
focal_zotu = ["Zotu27"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu104

In [ ]:
focal_zotu = ["Zotu104"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

##### Alternative zOTUs

In [ ]:
(
    d
    .loc[['100205']]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu167

In [ ]:
focal_zotu = ["Zotu167"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("cc", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu26

In [ ]:
focal_zotu = ["Zotu26"]

(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

In [ ]:
focal_zotu = ["Zotu100"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

### "Erysipelatoclostridiaceae" (Zotu46)

In [ ]:
focal_zotu = ["Zotu46"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Oscillospiraceae" (Zotu49)

In [ ]:
focal_zotu = ["Zotu49"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Erysipelotrichaceae" (Zotu34)

In [ ]:
focal_zotu = ["Zotu34"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Ruminococcaceae"

#### Zotu78

In [ ]:
focal_zotu = ["Zotu78"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu16

In [ ]:
focal_zotu = ["Zotu16"]
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

In [ ]:
motu_taxonomy.g__.loc['102040']

#### Zotu9

In [ ]:
focal_zotu = ["Zotu9"]
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

In [ ]:
motu_taxonomy.loc['103166']

### "Oscillospiraceae" (Zotu13)

In [ ]:
focal_zotu = ["Zotu13"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Bacteroidaceae"

####  Zotu74

In [ ]:
focal_zotu = ["Zotu74"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

##### Deeper dive into alternative Zotus to the same species

In [ ]:
(
    d
    .loc[['102549']]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

#### Zotu6

In [ ]:
focal_zotu = ["Zotu6"]
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

In [ ]:
motu_taxonomy[['g__', 's__']].loc['101346']

#### Zotu12

In [ ]:
focal_zotu = ["Zotu12"]
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

In [ ]:
motu_taxonomy[['g__', 's__']].loc['101337']

#### 102478

In [ ]:
focal_motu = '102478'

print(motu_taxonomy.loc[focal_motu])

(
    d.reorder_levels(["species_id", "zotu"])
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .join(rotu_taxonomy0.str[-20:])
    .loc[focal_motu]
)

#### 101378

In [ ]:
focal_motu = '101378'

print(motu_taxonomy.loc[focal_motu])

(
    d.reorder_levels(["species_id", "zotu"])
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .join(rotu_taxonomy0.str[-20:])
    .loc[focal_motu]
)

In [ ]:
motu_taxonomy.loc[focal_motu]

### "Peptostreptococcaceae" (Zotu100)

In [ ]:
focal_zotu = ["Zotu100"]
focal_family_list = rotu_taxonomy0.loc[focal_zotu].str.split(";").str[-3].unique()
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["zotu", "species_id"])
    .loc[focal_zotu]
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .head(10)
    .join(motu_taxonomy.s__)
)

### "Enterococcaceae" (100323)

In [ ]:
focal_motu = ["100323"]
focal_family_list = list(motu_taxonomy.loc[focal_motu].f__.str[3:].values)
focal_family_list
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["species_id", "zotu"])
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .join(rotu_taxonomy0.str[-20:])
    .loc[focal_motu]
)

### "f__Desulfovibrionaceae" (101359)

In [ ]:
focal_motu = ["101359"]
focal_family_list = list(motu_taxonomy.loc[focal_motu].f__.str[3:].values)
focal_family_list
assert len(focal_family_list) == 1
focal_family = focal_family_list[0]
print(focal_family)

In [ ]:
x_taxa = idxwhere(rotu_taxonomy0.str.contains(f";{focal_family};"))
y_taxa = idxwhere(
    motu_taxonomy.apply(lambda x: ";".join(x), axis=1).str.contains(
        f"f__{focal_family};"
    )
    & motu_taxonomy.index.to_series().isin(motu_rabund.columns)
)

In [ ]:
rotu_taxonomy0.loc[x_taxa].reset_index().values

In [ ]:
motu_taxonomy.loc[y_taxa]

In [ ]:
from sklearn.cross_decomposition import PLSCanonical, PLSRegression

x, y = align_indexes(rotu_rabund[x_taxa], motu_rabund[y_taxa])
pls = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=False, tol=1e-7).fit(
    x, y
)

In [ ]:
pseudo_map = pd.DataFrame(
    (pls.y_rotations_ @ pls.x_rotations_.T), index=y.columns, columns=x.columns
)
pseudo_map.shape

In [ ]:
plt.hist(pseudo_map.values.flatten(), bins=1000)
plt.yscale("log")
None

In [ ]:
cross_correlation = pd.DataFrame(
    1
    - sp.spatial.distance.cdist(
        x.T, y.T, metric=lambda x, y: 1 - sp.stats.pearsonr(x, y)[0]
    ),
    columns=y.columns,
    index=x.columns,
)

In [ ]:
sns.clustermap(
    pseudo_map.fillna(0).T, center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)
sns.clustermap(
    cross_correlation.fillna(0), center=0, figsize=(5, 5), xticklabels=1, yticklabels=1
)

In [ ]:
d = pd.DataFrame(
    dict(
        pls=pseudo_map.stack(),
        cc=cross_correlation.T.stack(),
        pls_rank_A=pseudo_map.rank(axis=0, ascending=False).stack(),
        pls_rank_B=pseudo_map.rank(axis=1, ascending=False).stack(),
        cc_rank_A=cross_correlation.T.rank(axis=0, ascending=False).stack(),
        cc_rank_B=cross_correlation.T.rank(axis=1, ascending=False).stack(),
    )
)

In [ ]:
(
    d.reorder_levels(["species_id", "zotu"])
    .join(rotu_rabund[x_taxa].mean().rename("rotu_mean"))
    .join(motu_rabund[y_taxa].mean().rename("motu_mean"))
    .assign(pls_score=lambda x: x.pls * x.rotu_mean * x.motu_mean)
    .sort_values("pls_score", ascending=False)
    .join(rotu_taxonomy0.str[-20:])
    .loc[focal_motu]
)

## All-by-all

In [ ]:
sample[lambda x: (x.sample_type == 'human') & (x.index.isin(motu_rabund.index))].shape[0]#.subject_id#.value_counts()

In [ ]:
sample[lambda x: (x.index.isin(motu_rabund.index)) & (x.subject_id != 'X')].shape[0]#.subject_id#.value_counts()

In [ ]:
sample[lambda x: (x.sample_type == 'human') & (x.index.isin(motu_rabund.index))].subject_id.unique().shape

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

raw_reciprocal_hits_results, _, _ = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    alpha=1,
)

reciprocal_hits_results = (
    raw_reciprocal_hits_results.join(motu_taxonomy[["f__", "g__", "s__"]], on="species_id")
    .join(rotu_taxonomy[["f__", "g__"]], lsuffix="mgen", rsuffix="zotu")
    .assign(
        passes_rank=lambda x: x["rank_product"] <= 2,
        passes_family=lambda x: x.f__mgen == 'f__' + x.f__zotu
    )
)

fig, ax = plt.subplots(figsize=(4.5, 6))
sns.kdeplot(reciprocal_hits_results['corr'], ax=ax, label='All', color='k', linestyle=':')
# sns.kdeplot(reciprocal_hits_results[lambda x: x.passes_rank]['corr'], ax=ax, label='Matches', color='k', linestyle='-')
ax.hist(reciprocal_hits_results[lambda x: x.passes_rank]['corr'], label='Matches', density=True)
ax.legend(loc='upper right')
ax.set_yticks([])
ax.set_ylabel('Density')
ax.set_xlabel('Correlation between\nzOTU and Species')

fig.savefig('fig/een_species_matching_correlation_density.pdf', bbox_inches='tight')

In [ ]:
reciprocal_hits_results[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

In [ ]:
reciprocal_hits_results[lambda x: x.passes_rank].reset_index().zotu.unique().shape

In [ ]:
sp.stats.mannwhitneyu(reciprocal_hits_results['corr'], reciprocal_hits_results[lambda x: x.passes_rank]['corr'])

In [ ]:
reciprocal_hits_results[lambda x: x.rank_product <= 2].rank_product.value_counts()

In [ ]:
# Matched zOTUs capture a median of 40% of relative abundance across all samples.
rotu_rabund[reciprocal_hits_results[lambda x: x.rank_product <= 2].reset_index().zotu.unique()].sum(1).median()

In [ ]:
reciprocal_hits_results.to_csv('fig/een_zotu_species_matching.tsv', sep='\t')

In [ ]:
reciprocal_hits_results.sort_values('species_id')

In [ ]:
d = pd.DataFrame(dict(rotu=rotu_rabund[['Zotu1', 'Zotu11']].sum(1), motu=motu_rabund[['102478']].sum(1))).dropna()
print(sp.stats.pearsonr(d['motu'], d['rotu']))
plt.scatter('rotu', 'motu', data=d)

In [ ]:
d = pd.DataFrame(dict(rotu=rotu_rabund[['Zotu4']].sum(1), motu=motu_rabund[['102506']].sum(1))).dropna()
print(sp.stats.pearsonr(d['motu'], d['rotu']))
plt.scatter('rotu', 'motu', data=d)

In [ ]:
# pair_type_palette={'Transition': 'plum', 'EEN': 'pink', 'PostEEN': 'lightblue'}


d = pd.DataFrame(dict(
    rotu=rotu_rabund[['Zotu5']].sum(1),
    motu_A=motu_rabund[['101386']].sum(1),
    motu_B=motu_rabund[['101493']].sum(1),
    motu_both=motu_rabund[['101493', '101386']].sum(1),

)).dropna()
print(sp.stats.pearsonr(d['motu_A'], d['rotu']))
print(sp.stats.pearsonr(d['motu_B'], d['rotu']))
print(sp.stats.pearsonr(d['motu_both'], d['rotu']))

# fig = plt.figure(figsize=(4, 3))
plt.scatter('rotu', 'motu_A', data=d, label='$\mathit{E. clostridioforme}$', s=40, alpha=0.7, edgecolor='grey', color='skyblue')
plt.scatter('rotu', 'motu_B', data=d, label='$\mathit{E. bolteae}$', s=40, alpha=0.7, edgecolor='grey', color='lightcoral')
plt.scatter('rotu', 'motu_both', data=d, label='Combined', s=40, alpha=0.7, edgecolor='grey', color='black')
plt.plot([0, 0.5], [0, 0.5])
plt.xlabel('zOTU5 relative abundance')
plt.ylabel('Species relative abundance')
plt.yscale('log')
plt.xscale('log')
plt.legend(loc='upper left', fontsize='small', markerscale=2)

plt.savefig('fig/een_zotu5_species_matching.pdf', bbox_inches='tight')

In [ ]:
d = pd.DataFrame(dict(
    rotu_A=rotu_rabund[['Zotu79']].sum(1),
    rotu_B=rotu_rabund[['Zotu80']].sum(1),
    rotu_both=rotu_rabund[['Zotu79', 'Zotu80']].sum(1),
    motu_A=motu_rabund[['100107']].sum(1),

)).dropna()
print(sp.stats.pearsonr(d['motu_A'], d['rotu_A']))
print(sp.stats.pearsonr(d['motu_A'], d['rotu_B']))
print(sp.stats.pearsonr(d['motu_A'], d['rotu_both']))
plt.scatter('motu_A', 'rotu_A', data=d, label='Zotu79')
plt.scatter('motu_A', 'rotu_B', data=d, label='Zotu80')
plt.scatter('motu_A', 'rotu_both', data=d, label='Combined')
plt.plot([0, 0.5], [0, 0.5])
plt.ylabel('zOTU relative abundance')
plt.xlabel('Species 100107 relative abundance')
# plt.yscale('log')
# plt.xscale('log')
plt.legend()

# plt.savefig('fig/een_zotu5_species_matching.pdf')

In [ ]:
d = pd.DataFrame(dict(
    rotu_A=rotu_rabund[['Zotu79']].sum(1),
    rotu_B=rotu_rabund[['Zotu80']].sum(1),
    rotu_both=rotu_rabund[['Zotu79', 'Zotu80']].sum(1),
    motu_A=motu_rabund[['100107']].sum(1),

)).dropna()
print(sp.stats.pearsonr(d['motu_A'], d['rotu_A']))
print(sp.stats.pearsonr(d['rotu_A'], d['rotu_B']))
plt.scatter('rotu_A', 'rotu_B', data=d)
plt.plot([0, 0.5], [0, 0.5])
plt.ylabel('Zotu80 relative abundance')
plt.xlabel('Zotu79 relative abundance')
# plt.yscale('log')
# plt.xscale('log')

# plt.savefig('fig/een_zotu5_species_matching.pdf')

## PLS-based

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

reciprocal_hits_results2, cc, aux = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=pls_pseudo_mapping,
    alpha=0.5,
)
reciprocal_hits_results2 = reciprocal_hits_results2.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')


reciprocal_hits_results2[lambda x: x.rank_product <= 2].sort_values('zotu')[lambda x: x.g__mgen == 'g__Faecalibacterium']#.xs('Zotu17', level='zotu')

In [ ]:
sns.heatmap(aux.x_rotations_)

In [ ]:
a = aux.x_rotations_
b = np.linalg.pinv(a).T

np.abs(a).max(), np.abs(b).max()

In [ ]:
sns.heatmap(aux.x_rotations_, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
sns.heatmap(np.linalg.pinv(aux.x_rotations_).T, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
sns.heatmap(np.linalg.pinv(aux.x_rotations_).T - aux.x_rotations_, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
sns.heatmap(aux.y_rotations_, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
sns.heatmap(np.linalg.pinv(aux.y_rotations_).T, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
sns.heatmap(np.linalg.pinv(aux.y_rotations_).T - aux.y_rotations_, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
x2y = (np.linalg.pinv(aux.x_rotations_).T @ aux.y_rotations_.T)

In [ ]:
y2x = (np.linalg.pinv(aux.y_rotations_).T @ aux.x_rotations_.T)

In [ ]:
sns.heatmap(aux.y_rotations_ @ aux.y_rotations_.T)

In [ ]:
sns.heatmap(aux.y_rotations_.T @ aux.y_rotations_)

In [ ]:
sns.heatmap(aux.x_rotations_ @ aux.x_rotations_.T)

In [ ]:
sns.heatmap(aux.x_rotations_.T @ aux.x_rotations_)

In [ ]:
sns.heatmap(x2y @ y2x)

In [ ]:
sns.heatmap(x2y, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
aux.coef_.shape

In [ ]:
sns.heatmap(aux.coef_.T, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm')

In [ ]:
(aux.x_rotations_ @ aux.y_rotations_.T)

In [ ]:
plt.yscale('log')
bins = np.linspace(-1, 1, num=201)
plt.hist(reciprocal_hits_results2['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results2[lambda x: x.rank_product <= 2]['corr'], bins=bins, label='hits')
plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <= 1].assign(
    g_match=lambda x: x.g__mgen.str[3:].str.split('_').str[0] == x.g__zotu,
    f_match=lambda x: x.f__mgen.str[3:].str.split('_').str[0] == x.f__zotu,
).f_match.sum()

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product == 1].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.g_match | x.f_match)]

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product == 2].shape

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product == 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

In [ ]:
from scipy import signal
sp.signal.correlate2d(motu_rabund, rotu_rabund)

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <= 2].sort_values('score')

In [ ]:
import networkx as nx

reciprocal_hits_results2[lambda x: x.rank_product <= 2].sort_values('score')

In [ ]:
g = nx.DiGraph()
g.add_edges_from(reciprocal_hits_results2[lambda x: x.rank_product <= 2].index)

fig, ax = plt.subplots(figsize=(40, 40))
# nx.draw_networkx(
#     g,
#     pos=nx.bipartite_layout(g, nodes=reciprocal_hits_results2.index.to_frame().species_id.values)
#     ax=ax,
# )
# nx.draw_networkx(
#     g,
#     pos=nx.spring_layout(
#         g,
#         pos=nx.bipartite_layout(g, nodes=reciprocal_hits_results2.index.to_frame().species_id.values)
#     ),
#     ax=ax,
# )
# nx.draw_networkx(
#     g,
#     pos=nx.planar_layout(g),
#     ax=ax,
# )
nx.draw_networkx(
    g,
    pos=nx.spring_layout(
        g,
        pos=nx.planar_layout(g),
    ),
    ax=ax,
)

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <=2].shape

In [ ]:
# Matched zOTUs capture a median of 40% of relative abundance across all samples.
rotu_rabund[reciprocal_hits_results2[lambda x: x.rank_product <= 2].reset_index().zotu.unique()].sum(1).median()

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

raw_reciprocal_hits_results, _, _ = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    alpha=1,
)

d = (
    reciprocal_hits_results2
    .assign(
        pearson=reciprocal_hits_results['corr'],  # Version #1
        passes_rank=lambda x: x["rank_product"] <= 2,
        passes_family=lambda x: x.f__mgen == 'f__' + x.f__zotu
    )
)

fig, ax = plt.subplots(figsize=(3, 4))
sns.kdeplot(d['corr'], ax=ax, label='All', color='k', linestyle=':')
ax.hist(d[lambda x: x.passes_rank]['pearson'], label='Matches')#, ax=ax, color='k', linestyle='-')

ax.legend(loc='upper right')
ax.set_yticks([])
ax.set_ylabel('Count / Density')
ax.set_xlabel('Pearson Correlation')

In [ ]:
reciprocal_hits_results2.sort_values('corr', ascending=False).head(50)

In [ ]:
reciprocal_hits_results2.xs('102327', level='species_id')

In [ ]:
reciprocal_hits_results2.xs('Zotu5', level='zotu').sort_values('corr', ascending=False)

In [ ]:
reciprocal_hits_results2.xs('Zotu75', level='zotu').sort_values('corr', ascending=False)

In [ ]:
d = pd.DataFrame(dict(
    x=motu_rabund[['102327']].sum(1),
    y=rotu_rabund[['Zotu103']].sum(1),
)).dropna()
print(sp.stats.pearsonr(d['x'], d['y']))
plt.scatter('x', 'y', data=d)
plt.plot([0, 0.5], [0, 0.5])
plt.yscale('log')
plt.xscale('log')

# plt.savefig('fig/een_zotu5_species_matching.pdf')

In [ ]:
reciprocal_hits_results2[lambda x: x['corr'] > 0.5].sort_values('species_id').head(50)

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <= 2].xs('Zotu5', level='zotu')

## PLS2-based

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

reciprocal_hits_results2, cc, aux = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=pls_coefs,
    alpha=0.5,
)
reciprocal_hits_results2 = reciprocal_hits_results2.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')


reciprocal_hits_results2[lambda x: x.rank_product <= 2].sort_values('zotu').rank_product.value_counts()

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

In [ ]:
family_mismatch_species_list = reciprocal_hits_results2[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)].index


# plt.yscale('log')
bins = np.linspace(-1, 1, num=101)
plt.hist(reciprocal_hits_results2['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results2[lambda x: x.rank_product <= 2]['corr'], bins=bins, label='hits')
plt.hist(reciprocal_hits_results2.loc[family_mismatch_species_list]['corr'], bins=bins, label='family-mismatch')

plt.ylim(0, 20)

plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

## PLS3-based

In [ ]:
def pls_symmetrical_coefs(x, y, scale=False, tol=1e-7, **kwargs):
    pls_xy = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=scale, tol=tol, max_iter=5000).fit(
        x.T, y.T
    )
    pls_yx = PLSCanonical(n_components=min(*x.shape, *y.shape), scale=scale, tol=tol, max_iter=5000).fit(
        y.T, x.T
    )
    coefs = pd.DataFrame(
        (pls_xy.coef_ + pls_yx.coef_.T), index=y.index, columns=x.index
    )
    return coefs.T, (pls_xy, pls_yx)

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

reciprocal_hits_results2, cc, aux = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=pls_symmetrical_coefs,
    alpha=0.5,
)
reciprocal_hits_results2 = reciprocal_hits_results2.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')


reciprocal_hits_results2[lambda x: x.rank_product <= 2].sort_values('zotu').rank_product.value_counts()

In [ ]:
reciprocal_hits_results2[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

In [ ]:
reciprocal_hits_results2.loc['102506'].sort_values('score')

In [ ]:
family_mismatch_species_list = reciprocal_hits_results2[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)].index


# plt.yscale('log')
bins = np.linspace(-1, 3, num=201)
plt.hist(reciprocal_hits_results2['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results2[lambda x: x.rank_product <= 2]['corr'], bins=bins, label='hits')
plt.hist(reciprocal_hits_results2.loc[family_mismatch_species_list]['corr'], bins=bins, label='family-mismatch')

plt.ylim(0, 20)

plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

In [ ]:
plt.yscale('log')
bins = np.linspace(-1, 3, num=201)
plt.hist(reciprocal_hits_results2['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results2[lambda x: x.rank_product <= 2]['corr'], bins=bins, label='hits')
plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

## PLS4-based

In [ ]:
PLSSVD.fit(

In [ ]:
PLSSVD().x_weights_

In [ ]:
from sklearn.cross_decomposition._pls import svd, svd_flip

def pls_svd_symmetrical_coefs2(x, y, centered=False, scaled=False, transpose=False, rcond=1e-10):
    # n_components = x.shape[0]

    if centered:
        x = x - x.mean(axis=0)
        y = y - y.mean(axis=0)
    if scaled:
        x = (x / np.sqrt((x**2).sum(axis=0))).dropna(axis=1)
        y = (y / np.sqrt((y**2).sum(axis=0))).dropna(axis=1)

        
    c = pd.DataFrame(np.dot(x.T, y), index=x.columns, columns=y.columns)
    if transpose:
        c = c.T

    u, s, vt = svd(c, full_matrices=False)

    max_s = s[0]
    u = u[:, s > (max_s * rcond)]
    vt = vt[s > (max_s * rcond)]
    u, vt = svd_flip(u, vt, u_based_decision=True)
    v = vt.T
    cc = pd.DataFrame(u @ v.T, index=c.index, columns=c.columns)
    if transpose:
        cc = cc.T
    return cc, (s, u, v)

def pls_svd_symmetrical_coefs3(x, y, scaled=False, transpose=False):
    if transpose:
        x, y = y, x
    pls = PLSSVD(n_components=min(*x.shape, *y.shape), scale=scaled).fit(x, y)
    cc = pd.DataFrame(pls.x_weights_ @ pls.y_weights_.T, index=x.columns, columns=y.columns)
    if transpose:
        cc = cc.T
    return cc, pls

def covar_pseudoinverse(x, y, centered=False, scaled=False):
    n_components = x.shape[0]
    if centered:
        x = x - x.mean(axis=0)
        y = y - y.mean(axis=0)
    if scaled:
        x = (x / np.sqrt((x**2).sum(axis=0))).dropna(axis=1)
        y = (y / np.sqrt((y**2).sum(axis=0))).dropna(axis=1)

        
    c = np.dot(x.T, y)
    c_pinv = np.linalg.pinv(c)
    # # u, s, vt = svd(c, full_matrices=False)
    # u = u[:, :n_components]
    # vt = vt[:n_components]
    # u, vt = svd_flip(u, vt, u_based_decision=True)
    # v = vt.T
    c_pinv = pd.DataFrame(c_pinv.T, index=x.columns, columns=y.columns)
    return c_pinv, None


def cross_covar(x, y, centered=False, scaled=False):
    n_components = x.shape[0]
    if centered:
        x = x - x.mean(axis=0)
        y = y - y.mean(axis=0)
    if scaled:
        x = (x / np.sqrt((x**2).sum(axis=0))).dropna(axis=1)
        y = (y / np.sqrt((y**2).sum(axis=0))).dropna(axis=1)

        
    c = np.dot(x.T, y)
    # c_pinv = np.linalg.pinv(c)
    # # # u, s, vt = svd(c, full_matrices=False)
    # # u = u[:, :n_components]
    # # vt = vt[:n_components]
    # # u, vt = svd_flip(u, vt, u_based_decision=True)
    # # v = vt.T
    c = pd.DataFrame(c, index=x.columns, columns=y.columns)
    return c, None


def reciprocal_hits(x, y, cc_func=lambda x, y: calc_cross_correlation(x, y, metric='correlation'), alpha=0.5):
    x, y = align_indexes(x, y)
    
    cc, aux = cc_func(x, y)
    
    cols = cc.columns
    idxs = cc.index

    x_weight = (x.mean().loc[idxs]) ** alpha
    y_weight = (y.mean().loc[cols]) ** alpha
    score = cc.multiply(x_weight, axis=0).multiply(y_weight, axis=1)
    
    cols_rank = score.rank(1, ascending=False)
    idxs_rank = score.rank(0, ascending=False)

    result = (cols_rank * idxs_rank).stack().to_frame('rank_product').assign(corr=cc.stack(), score=score.stack())
    # No longer filtering
    # [lambda x: (x.rank_product <= rank_thresh) & (x['corr'] >= corr_thresh)]
    
    return result, cc, aux

In [ ]:
x, y = motu_rabund.copy(), rotu_rabund.copy()
print(len(x), len(y))


# TODO: FIXME: Testing a permutation
# x.index = x.sample(frac=1, random_state=0).index

reciprocal_hits_results4, cc, aux = reciprocal_hits(
    x,
    y,
    cc_func=lambda x, y: pls_svd_symmetrical_coefs3(x, y, scaled=False, transpose=False),
    alpha=0.5,
)
reciprocal_hits_results4 = reciprocal_hits_results4.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')


reciprocal_hits_results4[lambda x: x.rank_product <= 2].sort_values('zotu').rank_product.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(cc, norm=mpl.colors.PowerNorm(1, vmin=-0.5, vmax=0.5), cmap='coolwarm', ax=ax)

In [ ]:
reciprocal_hits_results4[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
    matches=lambda x: (x.f_match | x.g_match),
).matches.mean()

In [ ]:
reciprocal_hits_results4.loc['101493'].sort_values('rank_product').head(2)

In [ ]:
reciprocal_hits_results4.xs('Zotu5', level='zotu').sort_values('rank_product').head(2)

In [ ]:
reciprocal_hits_results4[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

In [ ]:
family_mismatch = reciprocal_hits_results4[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)].index


# plt.yscale('log')
bins = np.linspace(-1, 1.5, num=201)
# bins = None
plt.hist(reciprocal_hits_results4['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results4[lambda x: x.rank_product == 1]['corr'], bins=bins, label='RBH')
plt.hist(reciprocal_hits_results4[lambda x: x.rank_product == 2]['corr'], bins=bins, label='Secondary')
# plt.hist(reciprocal_hits_results2.loc[family_mismatch_species_list]['corr'], bins=bins, label='family-mismatch')

plt.ylim(0, 100)

plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

In [ ]:
plt.yscale('log')
bins = np.linspace(-1, 1, num=201)
plt.hist(reciprocal_hits_results4['corr'], bins=bins, label='all')
plt.hist(reciprocal_hits_results4[lambda x: x.rank_product == 1]['corr'], bins=bins, label='Best')
plt.hist(reciprocal_hits_results4[lambda x: x.rank_product == 2]['corr'], bins=bins, label='Secondary')
plt.xlabel('Score')
plt.ylabel('count')
plt.legend()
None

In [ ]:
species_of_interest = [str(s) for s in [
101386,
100196,
102455,
101345,
101337,
100203,
101433,
101304,
102320,
102372,
100183,
100108,
101385,
100044,
100265,
102470,
100303,
101500,
101531,
102469,
101415,
100003,
102395,
102512,
100878,
101188,
100159,
100033,
100062,
100074,
100201,
100193,
100231,
102396,
102616,
101374,
100238,
102369,
103683,
101338,
100099,
102538,
102351,
100323,
103682,
100123,
102506,
102453,
]]

d = reciprocal_hits_results4.loc[species_of_interest].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: (x.rank_product <= 2)]

col_list = ['rank_product', 'corr', 'f_mgen_norm', 'g_mgen_norm', 'f_zotu_norm', 'g_zotu_norm', 'f_match', 'g_match']
d[col_list].to_csv('species_zotu_pls4_matching.tsv', sep='\t')
d[col_list]

# Cosine-dissimilarity based

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

reciprocal_hits_results3, cc, aux = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=lambda x, y: calc_cross_correlation(x, y, metric='cosine'),
    alpha=0.5,
)
reciprocal_hits_results3 = reciprocal_hits_results3.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')


reciprocal_hits_results3[lambda x: x.rank_product <= 2].sort_values('zotu').rank_product.value_counts()

In [ ]:
reciprocal_hits_results3[lambda x: x.rank_product <= 2].assign(
    g_mgen_norm=lambda x: x.g__mgen.str[3:].str.split('_').str[0],
    g_zotu_norm=lambda x: x.g__zotu.str.split('_').str[0],
    g_match=lambda x: x.g_mgen_norm == x.g_zotu_norm,
    f_mgen_norm=lambda x: x.f__mgen.str[3:].str.split('_').str[0],
    f_zotu_norm=lambda x: x.f__zotu.str.split('_').str[0],
    f_match=lambda x: x.f_mgen_norm == x.f_zotu_norm,
)[lambda x: ~(x.f_match | x.g_match)]

## Non-negative PLS?

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

x, y = lib.pandas_util.align_indexes(motu_rabund, rotu_rabund)

sns.clustermap((x.T @ y), norm=mpl.colors.SymLogNorm(1e-5))

In [ ]:
from sklearn.decomposition import non_negative_factorization
z = (x.T @ y)
w, h, n_iter = non_negative_factorization(z, n_components=max(*z.shape), verbose=2, solver='mu', alpha_W=0.1, alpha_H=0.0, l1_ratio=1.0)

In [ ]:
w.shape

In [ ]:
sns.clustermap(w, norm=mpl.colors.SymLogNorm(1e-10))

## Cross-covariance-based

In [ ]:
motu_list = motu_rabund.columns
rotu_list = rotu_rabund.columns
print(len(motu_list), len(rotu_list))

reciprocal_hits_results3, cc, aux = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=cross_covar,
    alpha=0.5,
)
reciprocal_hits_results3 = reciprocal_hits_results3.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')

reciprocal_hits_results3[lambda x: x.rank_product <= 2].sort_values('zotu')#[lambda x: x.g__mgen == 'g__Faecalibacterium']#.xs('Zotu17', level='zotu')

## Family-wise

In [ ]:
family = 'Lachnospiraceae'

motu_list = idxwhere(motu_taxonomy.f__.str.startswith(f'f__{family}'))
rotu_list = idxwhere(rotu_taxonomy.f__.str.startswith(f'{family}'))

print(len(motu_list), len(rotu_list))

In [ ]:
rotu_taxonomy.f__.unique()

In [ ]:
reciprocal_hits_results3 = []
# for family in ['Clostridiaceae']:
for family in rotu_taxonomy.f__.unique():
    if family == '':
        continue
    motu_list = idxwhere(motu_taxonomy.f__.str.startswith(f'f__{family}'))
    rotu_list = idxwhere(rotu_taxonomy.f__.str.startswith(f'{family}'))
    print(family, len(motu_list), len(rotu_list))
    reciprocal_hits_results3.append(reciprocal_hits(
        motu_rabund.reindex(columns=motu_list).dropna(axis='columns'),
        rotu_rabund.reindex(columns=rotu_list).dropna(axis='columns'),
        cc_func=calc_cross_correlation,
        # rank_thresh=2,
        alpha=1,
    ))
    
    
reciprocal_hits_results3 = pd.concat(reciprocal_hits_results3).join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')
reciprocal_hits_results3.sort_values('species_id')

In [ ]:
reciprocal_hits_results3[lambda x: x['corr'] > 0.5].sort_values('species_id')

In [ ]:
plt.hist(reciprocal_hits_results3['corr'])

In [ ]:
d = pd.DataFrame(dict(rotu=rotu_rabund[['Zotu10', 'Zotu15']].sum(1), motu=motu_rabund[['100032']].sum(1))).dropna()
print(sp.stats.pearsonr(d['motu'], d['rotu']))
plt.scatter('rotu', 'motu', data=d)
plt.plot([0, 0.5], [0, 0.5])

In [ ]:
reciprocal_hits_results4, _, _ = reciprocal_hits(
    motu_rabund,
    rotu_rabund,
    cc_func=calc_cross_correlation,
    # rank_thresh=2,
    alpha=1,
)
    
    
reciprocal_hits_results3 = reciprocal_hits_results4.join(motu_taxonomy[['f__', 'g__', 's__']], on='species_id').join(rotu_taxonomy[['f__', 'g__']], lsuffix='mgen', rsuffix='zotu').sort_values('zotu')
reciprocal_hits_results3.sort_values('species_id')